In [1]:
import pandas as pd
import numpy as np

import time
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

In [2]:
data = pd.read_csv('data_non_linear.csv')
data.shape

(1348099, 105)

In [8]:
data['loan_status'].value_counts()

1    1078739
0     269360
Name: loan_status, dtype: int64

In [5]:
pd.set_option('display.max_columns', None)
data.head()

,Unnamed: 0,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,issue_year,term_code,grade_code,sub_grade_code,emp_title_code,home_ownership_code,verification_status_code,purpose_code,title_code,earliest_cr_line_code,initial_list_status_code,last_pymnt_d_code,next_pymnt_d_code,last_credit_pull_d_code,application_type_code,disbursement_method_code,debt_settlement_flag_code,emp_length_code
0,0,3600.0,13.99,123.03,55000.0,1,5.91,0.0,675.0,679.0,1.0,30.0,7.0,0.0,2765.0,29.7,13.0,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,122.67,564.0,560.0,0.0,30.0,1.0,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,2015,0.0,2.0,13.0,352908.0,1.0,0.0,2.0,16586.0,108.0,1.0,57.0,8.0,94.0,0.0,0.0,0.0,1.0
1,1,24700.0,11.99,820.28,65000.0,1,16.06,1.0,715.0,719.0,4.0,6.0,22.0,0.0,21470.0,19.2,38.0,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,926.35,699.0,695.0,0.0,44.0,1.0,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,37.0,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,2015,0.0,2.0,10.0,103053.0,1.0,0.0,11.0,5247.0,165.0,1.0,77.0,8.0,94.0,0.0,0.0,0.0,1.0
2,2,20000.0,10.78,432.66,63000.0,1,10.78,0.0,695.0,699.0,0.0,31.0,6.0,0.0,7869.0,56.2,18.0,0.0,0.0,22705.924294,22705.92,20000.0,2705.92,0.0,0.0,0.0,15813.30,704.0,700.0,0.0,44.0,1.0,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,73.0,0.0,2.0,2081.0,65.0,14000.0,2.0,5.0,1.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,37.0,10.0,33.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,2015,1.0,1.0,8.0,376427.0,1.0,0.0,4.0,16586.0,105.0,1.0,78.0,8.0,94.0,1.0,0.0,0.0,1.0
3,3,10400.0,22.45,289.91,104433.0,1,25.37,1.0,695.0,699.0,3.0,12.0,12.0,0.0,21929.0,64.5,35.0,0.0,0.0,11740.500000,11740.50,10400.0,1340.50,0.0,0.0,0.0,10128.96,704.0,700.0,0.0,44.0,1.0,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,2015,1.0,5.0,25.0,67871.0,1.0,1.0,6.0,31101.0,414.0,1.0,66.0,8.0,93.0,0.0,0.0,0.0,3.0
4,4,11950.0,13.44,405.18,34000.0,1,10.20,0.0,690.0,694.0,0.0,31.0,5.0,0.0,8822.0,68.4,6.0,0.0,0.0,13708.948530,13708.95,11950.0,1758.95,0.0,0.0,0.0,7653.56,759.0,755.0,0.0,44.0,1.0,0.0,0.0,12798.0,0

In [6]:
data = data.drop(['Unnamed: 0'], axis = 1)
data.describe()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,issue_year,term_code,grade_code,sub_grade_code,emp_title_code,home_ownership_code,verification_status_code,purpose_code,title_code,earliest_cr_line_code,initial_list_status_code,last_pymnt_d_code,next_pymnt_d_code,last_credit_pull_d_code,application_type_code,disbursement_method_code,debt_settlement_flag_code,emp_length_code
count,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1348099.0,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06,1.348099e+06
mean,1.440900e+04,1.324156e+01,4.377778e+02,7.623771e+04,8.001927e-01,1.827413e+01,3.176258e-01,6.961622e+02,7.001624e+02,6.621895e-01,3.263311e+01,1.159043e+01,2.150450e-01,1.627072e+04,5.181100e+01,2.497592e+01,2.840541e-01,2.840541e-01,1.481171e+04,1.478309e+04,1.217126e+04,2.397581e+03,1.587746e+00,2.412753e+02,4.021709e+01,5.415757e+03,6.772483e+02,6.601138e+02,1.709889e-02,4.392159e+01,1.0,5.038206e-03,2.363829e+02,1.379068e+05,1.020160e+00,2.316073e+00,3.096241e-01,1.296217e+00,1.555974e+01,2.820576e+04,7.180065e+01,1.167850e+00,2.400846e+00,4.861515e+03,5.807023e+01,3.240553e+04,1.037206e+00,6.417741e-01,1.523396e+00,4.667179e+00,1.316769e+04,9.961389e+03,5.995176e+01,9.043846e-03,1.486922e+01,1.261346e+02,1.805644e+02,1.284913e+01,7.758063e+00,1.645875e+00,2.335918e+01,3.762766e+01,6.488468e+00,3.393608e+01,4.830016e-01,3.609227e+00,5.609923e+00,4.702312e+00,8.043680e+00,8.434011e+00,8.212170e+00,1.447420e+01,5.561744e+00,1.161129e+01,7.469778e-04,3.240118e-03,8.424826e-02,2.169567e+00,9.446955e+01

In [7]:
Target = data['loan_status']
Features = data.drop(['loan_status'], axis = 1)

In [40]:
# Under sampling the imbalanced dataset
undersample = RandomUnderSampler(sampling_strategy = 0.5, random_state = 42)
features_balanced, target_balanced = undersample.fit_resample(Features, Target)
print(features_balanced.shape)
print(target_balanced.shape)

(808080, 103)
(808080,)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(features_balanced, target_balanced, test_size = 0.3, random_state = 42)

In [28]:
KF = KFold(3, shuffle = True, random_state= 42)

# Random Forest

In [42]:
start1 = time.time()
forest = RandomForestClassifier(n_jobs = -1, oob_score = True, 
                                random_state = 42, class_weight= "balanced").fit(X_train, y_train)
end1 = time.time()
Time1 = end1 - start1
print(Time1)

254.60912919044495


In [43]:
train_scores1 = cross_val_score(forest, X_train, y_train, cv = KF)
test_scores1 = cross_val_score(forest, X_test, y_test, cv = KF)

train_ac1 = round(forest.score(X_train, y_train), 4)
test_ac1 = round(forest.score(X_test, y_test), 4)
recall1 = round(recall_score(y_test, forest.predict(X_test)), 4)
roc1 = round(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1]), 4)

print(train_scores1)
print(test_scores1)
print(train_ac1)
print(test_ac1)
print(recall1)
print(roc1)

[0.99641478 0.99672239 0.99621325]
[0.99560687 0.99507475 0.99591625]
1.0
0.9969
0.9997
0.9998


In [56]:
forest.score(X_train, y_train)

0.9999964642821786

In [46]:
confusion_matrix(y_test, forest.predict(X_test))

array([[ 80047,    712],
       [    45, 161620]], dtype=int64)

# Gradient Boosting

In [48]:
start2 = time.time()
boosting = GradientBoostingClassifier(random_state = 42).fit(X_train, y_train)
end2 = time.time()
Time2 = end2 - start2
print(Time2)

1113.1691370010376


In [49]:
train_scores2 = cross_val_score(boosting, X_train, y_train, cv = KF)
test_scores2 = cross_val_score(boosting, X_test, y_test, cv = KF)

train_ac2 = round(boosting.score(X_train, y_train), 4)
test_ac2 = round(boosting.score(X_test, y_test), 4)
recall2 = round(recall_score(y_test, boosting.predict(X_test)), 4)
roc2 = round(roc_auc_score(y_test, boosting.predict_proba(X_test)[:, 1]), 4)

print(train_scores2)
print(test_scores2)
print(train_ac2)
print(test_ac2)
print(recall2)
print(roc2)

[0.99586851 0.99608596 0.99603292]
[0.99605237 0.995347   0.99629987]
0.9962
0.9962
0.9998
0.9997


In [52]:
confusion_matrix(y_test, forest.predict(X_test))

array([[ 80047,    712],
       [    45, 161620]], dtype=int64)

In [61]:
df = pd.DataFrame([['Random Forest', train_ac1, test_ac1, recall1, roc1],
                  ['Gradient Boosting', train_ac2, test_ac2, recall2, roc2]], 
                 columns=['model', 'train score','test score','recall score', 'ROC AUC score'])
df

,model,train score,test score,recall score,ROC AUC score
0,Random Forest,1.0000,0.9969,0.9997,0.9998
1,Gradient Boosting,0.9962,0.9962,0.9998,0.9997


In [62]:
df2 = pd.read_csv('result.csv')

In [65]:
result = pd.concat([df, df2], ignore_index=True)
result.drop(['Unnamed: 0'], axis = 1, inplace = True)
result

,model,train score,test score,recall score,ROC AUC score
0,Random Forest,1.0000,0.9969,0.9997,0.9998
1,Gradient Boosting,0.9962,0.9962,0.9998,0.9997
2,Logit,0.9310,0.9315,0.9287,0.9822
3,SVM,0.9144,0.7340,0.8888,0.5546
4,Naive Bayes,0.9073,0.9069,0.9610,0.9539
